# Лекция №6. Функциональное программирование

П.Н. Советов, РТУ МИРЭА

## Введение

До сих пор мы, в основном, явно указывали в программах на Питоне последовательность действий, содержащих присваивания и изменяемые данные. Это соответствует императивному подходу к программированию.

В функциональном программировании, в противовес программированию императивному, основной операцией является применение функций к своим аргументам. Такие функции являются "чистыми" в математическом смысле, то есть результат их выполнения полностью определяется входными аргументами. В противоположность чистым функциям, вычисление функции, обладающей побочным эффектом, зависит неявным образом от окружения. Например, функция random из одноименного модуля стандартной библиотеки Питона использует в своей работе глобальную переменную.

При анализе императивной программы программисту приходится в уме пошагово имитировать работу компьютера. В функциональном программировании используется более простая, чем в императивном программировании, подстановочная модель вычислений (выражения заменяются их результатами), которой программист может пользоваться умозрительно с большим удобством.

Важным свойством программ, которое обеспечивается функциональным программированием, является компонуемость или сочетаемость — сложные программы строятся, свободно используя простые строительные блоки-функции, в духе блоков игрового конструктора. В Питоне компонуемость не всегда выдерживается. К примеру, мы не можем поместить обычный for или if внутрь некоторого выражения. Конечно, мы могли бы использовать списковое включение или тернарный if, но эти конструкции обладают рядом ограничений, то есть плохо компонуются.

Остается вопрос, каким образом функциональные программы, без присваиваний и изменяемых данных, могут делать что-то полезное. При использовании функционального стиля программирования в таком языке, как Питон, предпочтительной является типовая архитектура системы под названием "функциональное ядро и императивная оболочка" (Functional core, imperative shell):

1. Ключевые вычислительные задачи и задачи преобразования данных формируют функциональное ядро системы. Здесь применяются только чистые функции и неизменяемые типы данных.
1. Работа с внешней средой, ввод/вывод, вспомогательные задачи организуются с помощью императивной оболочки.

## Еще раз о пространствах имен и функциях

В Питоне используется несколько пространств имен, вложенных друг в друга:

1. Пространство имен модуля (globals), которое включает в себя пространство имен встроенных функций (builtins).
1. Пространство имен объемлющих функций (nonlocal).
1. Пространство имен текущей функции (locals).

Переменная в функции может относиться только к одному из этих пространств имен. Рассмотрим следующий пример:

In [1]:
a = 42

def f():
    print(a)
    a = 1
    
# Раскомментируйте, чтобы получить ошибку
# f()

Компилятор Питона при трансляции функции руководствуется следующими правилами:
1. Если `a` не используется в левой части присваивания, а также отсутствует в объемлющих функциях, то `a` трактуется, как элемент из globals.
1. Если `a` не используется в левой части присваивания, но присутствует в объемлющей функции, то формируется nonlocal-ссылка на `a` из объемлющей функции.
1. Если `a` присваивается некоторое значение, то `a` является элементом locals.

В примере выше `a` трактуется, как локальная переменная, поэтому вызов print до присваивания приводит к ошибке. Разумеется, с помощью global или nonlocal (о нем будет сказано далее) можно явно указать, к какому пространству имен принадлежит `a`:

In [43]:
a = 42

def f():
    global a
    print(a)
    a = 1

f()

42


## Функции высшего порядка

В Питоне функции являются объектами первого класса. Это означает, что функции можно использовать, как аргументы, возвращать из функций, а также записывать в переменные. Это позволяет использовать важнейший элемент функционального программирования — функции высшего порядка.

Функция высшего порядка (ФВП), помимо значений других типов, использует функции в качестве своих аргументов/результата. В математике известны примеры ФВП. Это, например, производные и интегралы. В современном программировании без ФВП тоже трудно обойтись: даже на языке C стандартная функция сортировки qsort требует ссылки на функцию в качестве одного из своих аргументов.

Ниже показан пример ФВП, которая принимает на вход функцию:

In [2]:
def f(func, arg):
    return func(arg)
    
def incr(x):
    return x + 1
    
print(f(incr, 10))

11


Пример ФВП, возвращающей функцию:

In [3]:
def f(x):
    def func(y):
        return y * x
    return func

g = f(10)
[g(x) for x in range(10)]

[0, 10, 20, 30, 40, 50, 60, 70, 80, 90]

Наконец, пример ФВП, получающей и возвращающей функцию:

In [4]:
def f(g, x):
    def func(size):
        return ' '.join([g(x) for i in range(size)])
    return func

def f1(x):
    return '*' * x

f2 = f(f1, 2)
f2(10)

'** ** ** ** ** ** ** ** ** **'

### Замыкания

Мы уже видели, что вложенные функции в Питоне могут использовать не только аргументы и локальные переменные, но также и свободные переменные — имена из объемлющих функций. Это возможно благодаря механизму лексического замыкания, при котором формируется специальный объект, включающий себя как ссылку на функцию, так и данные, которые функция использует из объемлющих функций.

Рассмотрим следующий пример:

In [5]:
def make_adder(x):
    def adder(y):
        # Замыкание, включающее в себя свободную переменную x
        return x + y
    return adder
    
f = make_adder(10)
print(f(5))

# Переменные adder
print(f.__code__.co_varnames)
# Свободные переменные adder
print(f.__code__.co_freevars)
# Значение x из захваченного окружения
print(f.__closure__[0].cell_contents)

15
('y',)
('x',)
10


Для модификации захваченной свободной переменной необходимо явно указать с помощью ключевого слова nonlocal, что модифицируемая переменная не является локальной или глобальной, а относится к одной из объемлющих функций:

In [6]:
def make_counter(n):
    i = 0
    def counter():
        nonlocal i # Необходимое указание, поскольку далее используется присваивание
        i = i - 1 if i > 0 else n
        return i
    return counter

counter = make_counter(2)

for i in range(16):
    print(counter(), end=' ')

2 1 0 2 1 0 2 1 0 2 1 0 2 1 0 2 

### Анонимные функции

Функциональный стиль программирования предполагает создание большого числа ФВП. При этом часто приходится создавать небольшие функции-аргументы, нужные только для одной сиюминутной задачи. В подобных случаях удобнее воспользоваться механизмом создания анонимных функций с помощью ключевого слова lambda.

Определение

```Python
func = lambda аргументы: выражение
```

эквивалентно следующему:

```Python
def func(аргументы):
    return выражение
```

Лямбда-выражение можно передать на вход или возвратить из ФВП, как показано ниже:

In [7]:
def f(g, x):
    return lambda size: ' '.join([g(x) for i in range(size)])

f(lambda x: '*' * x, 2)(10)

'** ** ** ** ** ** ** ** ** **'

Еще один пример:

In [8]:
def twice(f):
    def g(x):
        return f(f(x))
    return g

twice(twice(lambda x: x * x))(2)

65536

Обратите внимание на ограничения lambda. В ее теле могут использоваться только те конструкции, которые доступны в качестве выражения, возвращаемого с помощью return.

Классическими в функциональном программировании являются функции map, filter и reduce. Они реализованы в Питоне, но для целей более близкого знакомства с ФВП далее приводится упрощенная реализация каждой их них.

Функция map принимает на вход унарную функцию и список, а возвращает новый список, элементы которого являются результатами применения функции-аргумента к элементам исходного списка:

In [9]:
def my_map(func, lst):
    res = []
    for x in lst:
        res.append(func(x))
    return res

my_map(lambda x: x * x, range(10))

[0, 1, 4, 9, 16, 25, 36, 49, 64, 81]

Функция filter принимает на вход унарный предикат и список, а возвращает новый список, элементы которого удовлетворяют предикату:

In [10]:
def my_filter(pred, lst):
    res = []
    for x in lst:
        if pred(x):
            res.append(x)
    return res

my_filter(lambda x: x % 2, range(10))

[1, 3, 5, 7, 9]

Функция reduce принимает на вход унарную функцию и список, последовательно применяет функцию-аргумент, сводя список до единственного значения:

In [11]:
def my_reduce(func, lst):
    x = lst[0]
    for y in lst[1:]:
        x = func(x, y)
    return x

my_reduce(lambda x, y: x + y, range(10))

45

Вместо функций map и filter в Питоне предпочтительнее использовать уже знакомые нам списковые включения, а также генераторы списков, о которых будет говориться далее.

### Декораторы

Декораторы синтаксически представляют собой аннотации, которые указываются перед аннотируемой функцией. Реализуются они с помощью ФВП, которая принимает на вход функцию и возвращает функцию.

Декоратор функции выглядит следующим образом:

```Python
@decorator
def f():
    print('...')
```    

Что эквивалентно выражению

```Python
def f():
    print('...')

f = decorator(f)
```

Простой пример:

In [44]:
def decorator(f):
    print('Декоратор сработал.')
    return f

@decorator
def f():
    print('...')

print('Запуск функции.')
f()

Декоратор сработал.
Запуск функции.
...


Еще один пример, с использованием ранее определенной функции twice, уже готовой к использованию в качестве декоратора:

In [12]:
@twice
def f(x):
    return x * x

f(f(2))

65536

Пример с регистрацией вызовов функции:

In [13]:
def call_tracer(f):
    def g(x):
        print('Запуск функции {}({})'.format(f.__name__, x))
        return f(x)
    return g

@call_tracer
def fib(n):
    if n == 0:
        return 0
    elif n == 1:
        return 1
    else:
        return fib(n - 1) + fib(n - 2)

fib(4)

Запуск функции fib(4)
Запуск функции fib(3)
Запуск функции fib(2)
Запуск функции fib(1)
Запуск функции fib(0)
Запуск функции fib(1)
Запуск функции fib(2)
Запуск функции fib(1)
Запуск функции fib(0)


3

Декораторы могут иметь параметры, как показано в примере ниже:

In [14]:
def decorator(x):
    def func(f):
        print(x)
        return f
    return func
    
@decorator('Запуск')
def my_func(x):
    print(x)
    
my_func(10)

Запуск
10


Помимо декораторов функции существуют также декораторы класса, которые принимают на вход класс и возвращают класс.

## Генераторы

Генераторы представляют собой особый вид функций, внутри которых находится оператор yield. Этот оператор приостанавливает вычисления внутри функции, сохраняет текущее состояние этих вычислений и, аналогично return, осуществляет возврат в вызывающую функцию. При этом вычисления в генераторе можно возобновить сразу после последнего вызова yield.

```Python
def func():
    ...
    yield выражение
    ...
```

Вызов функции-генератора осуществляется особым способом, с помощью функции next, как показано в примере ниже:

In [15]:
def countup(n):
    print('Начало цикла:')
    for i in range(n):
        yield i
        print('Следующая итерация:')

# Создан объект генератора, ему передан аргумент,
# но countup еще не начал работу
c = countup(4)
print(type(c))
for i in range(4):
    print(next(c)) # выполнение останавливается после yield i
    print('...')
try:
    print(next(c))
except StopIteration:
    print('Выход из генератора по исключению')


<class 'generator'>
Начало цикла:
0
...
Следующая итерация:
1
...
Следующая итерация:
2
...
Следующая итерация:
3
...
Следующая итерация:
Выход из генератора по исключению


Генераторы поддерживают протокол итераторов:

In [16]:
for i in countup(4):
    print(i, end=' ')

Начало цикла:
0 Следующая итерация:
1 Следующая итерация:
2 Следующая итерация:
3 Следующая итерация:


In [22]:
def gen():
    for i in range(10):
        yield i

list(gen())

[0, 1, 2, 3, 4, 5, 6, 7, 8, 9]

В следующем примере генерируются все возможные пароли, символы которых принадлежат некоторому алфавиту. Использование генератора позволяет упростить код и сделать его производительным, поскольку порождение паролей происходит по необходимости, без создания лишних структур данных.

In [17]:
def gen_password(abc):
    for a in abc:
        for b in abc:
            for c in abc:
                for d in abc:
                    yield f'{a}{b}{c}{d}'


for p in gen_password('absp'):
    if p == 'pass':
        print('Пароль найден')
        break

Пароль найден


Предположим, что мы хотим реализовать следующие вычисления:

In [27]:
import math

my_filter(lambda x: x >= 0, my_map(math.sin, list(range(10000))))[:10]

[0.0,
 0.8414709848078965,
 0.9092974268256817,
 0.1411200080598672,
 0.6569865987187891,
 0.9893582466233818,
 0.4121184852417566,
 0.4201670368266409,
 0.9906073556948704,
 0.6502878401571168]

Здесь формируется большое количество промежуточных данных, от которых можно избавиться, если реализовать вычисления "по необходимости", с помощью генераторов.

Ниже показан более эффективный вариант решения той же задачи с помощью генераторов:

In [28]:
# Сгенерировать бесконечный поток чисел 0, 1, 2, ...
def gen_count():
    i = 0
    while True:
        yield i
        i += 1

def gen_map(func, gen):
    for x in gen:
        yield func(x)

def gen_filter(pred, gen):
    for x in gen:
        if pred(x):
            yield x

# Возвратить список n первых сгенерированных элементов
def take(n, gen):
    return [next(gen) for i in range(n)]

take(10, gen_filter(lambda x: x >= 0, gen_map(math.sin, gen_count())))

[0.0,
 0.8414709848078965,
 0.9092974268256817,
 0.1411200080598672,
 0.6569865987187891,
 0.9893582466233818,
 0.4121184852417566,
 0.4201670368266409,
 0.9906073556948704,
 0.6502878401571168]

В Питоне существуют генераторы-выражения, которые действуют аналогично уже изученным нами списковым включениям, но явно не порождают список. Генераторы-выражения представляют собой простой синтаксис для описания небольших генераторов:

In [37]:
lst = [i * i for i in range(10)]
gen = (i * i for i in range(10)) # Генератор-выражение
print(lst, gen, list(gen))

[0, 1, 4, 9, 16, 25, 36, 49, 64, 81] <generator object <genexpr> at 0x00000165841F3970> [0, 1, 4, 9, 16, 25, 36, 49, 64, 81]


In [41]:
print(sum((i ** 3 for i in range(10))))
print(sum(i ** 3 for i in range(10))) # Скобки необязательны

2025
2025


## Неизменяемые типы данных

В Питоне существует аналог изменяемого массива — уже хорошо знакомый нам кортеж. Действительно, элементы кортежа нельзя изменить:

In [11]:
t1 = (1, 2, 3)
t2 = ('a', ['b', 'c'])
# t1[2] = 42 Не сработает
t2[1][0] = 50 # Сработает, поскольку ссылка на список не изменилась
print(t2)

def replace(tup, index, val):
    '''
    Замена элемента кортежа в функциональном стиле.
    Возращается новый кортеж.
    '''
    return tup[:index] + (val,) + tup[index + 1:]

replace(t1, 2, 42)

('a', [50, 'c'])


(1, 2, 42)

Множество в Питоне является изменяемой структурой данных, поэтому не может, в частности, выступать в качестве ключа словаря. Имеется неизменяемый вариант множества — frozenset:

In [16]:
s1 = set(['a', 'b', 'c'])
d = {}
# d[s1] = 42 Не сработает
s2 = frozenset(['a', 'b', 'c'])
d[s2] = 42
d

{frozenset({'a', 'b', 'c'}): 42}

В качестве неполного аналога словаря в Питоне выступает структура данных namedtuple (именованный кортеж):

In [24]:
from collections import namedtuple

Person = namedtuple('Person', 'name age')
p1 = Person(name='Ivan', age=20)
print(p1.name, p1.age)
# p1.age += 1 Не сработает
print(p1._replace(age=p1.age + 1)) # Возвращен новый именованный кортеж

Ivan 20
Person(name='Ivan', age=21)


## Модуль functools

Модуль functools содержит ряд полезных определений, которые используются для функций высшего порядка. В частности, там имеется уже рассмотренная выше функция reduce.

Декоратор lru_cache осуществляет запоминание (мемоизацию) вычислений чистой функции. Если функция была вызвана повторно с теми же аргументами, то результат будет взят из таблицы:

In [47]:
from timeit import timeit
from functools import lru_cache

fact1 = lambda n: n * fact(n - 1) if n else 1

@lru_cache
def fact2(n):
    return n * fact(n - 1) if n else 1

print(timeit(lambda: fact1(1000), number=1000))
print(timeit(lambda: fact2(1000), number=1000))

0.5376468999997996
0.0007041000003482623


Функция partial принимает на вход функцию и возвращает новую функцию, некоторые аргументы которой уже содержат конкретные значения:

In [50]:
from functools import partial

def power(x, n):
    return x ** n

power2 = partial(power, n=2)
# Эквивалентно power2 = lambda x: power(x, n=2)

print(power2(10))

100


## Модуль itertools

Модуль itertools предназначен для работы с итераторами (генераторами) в высокопроизводительном функциональном стиле.

Итератор chain группирует итераторы-аргументы для последовательного выполнения:

In [57]:
from itertools import chain

for i in chain(range(1, 5), "abcd", range(10, 15)):
    print(i, end=' ')

1 2 3 4 a b c d 10 11 12 13 14 

В itertools имеется ряд итераторов для операций из области комбинаторики:

In [63]:
from itertools import product

for i in product('abcd', repeat=2):
    print(i)

('a', 'a')
('a', 'b')
('a', 'c')
('a', 'd')
('b', 'a')
('b', 'b')
('b', 'c')
('b', 'd')
('c', 'a')
('c', 'b')
('c', 'c')
('c', 'd')
('d', 'a')
('d', 'b')
('d', 'c')
('d', 'd')


In [61]:
from itertools import combinations

for i in combinations('abcd', 3):
    print(i)

('a', 'b', 'c')
('a', 'b', 'd')
('a', 'c', 'd')
('b', 'c', 'd')


In [65]:
from itertools import permutations

for i in permutations('abcd', 2):
    print(i)

('a', 'b')
('a', 'c')
('a', 'd')
('b', 'a')
('b', 'c')
('b', 'd')
('c', 'a')
('c', 'b')
('c', 'd')
('d', 'a')
('d', 'b')
('d', 'c')


Итератор count является расширенной версией реализованной выше функции gen_count, а islice позволяет сделать срез для итератора:

In [71]:
from itertools import count, islice

list(islice((i for i in count(5)), 0, 10))

[5, 6, 7, 8, 9, 10, 11, 12, 13, 14]

Итератор groupby последовательно объединяет в группы элементы итератора-аргумента по заданному ключу.

In [85]:
from itertools import groupby

# Ключом по умолчанию является сам элемент
print([(k, len(list(g))) for k, g in groupby('AAAABBBCCDAABBB')])

# Ключом является len
print([(k, list(g)) for k, g in groupby(['one', 'one', 'two', '3', '3', '3'], key=len)])

[('A', 4), ('B', 3), ('C', 2), ('D', 1), ('A', 2), ('B', 3)]
[(3, ['one', 'one', 'two']), (1, ['3', '3', '3'])]


## Литература

Для продолжения знакомства с функциональным программированием рекомендуются следующие фундаментальные учебники:

1. Абельсон Харольд, Сассман Джеральд Джей. "Структура и Интерпретация Компьютерных Программ".
1. Пирс Бенджамин. "Типы в языках программирования".
1. Окасаки Крис. "Чисто функциональные структуры данных".